In [1]:
import numpy as np
import tensorflow as tf

from utils import x_generator
from utils.arithmetic import *
from utils.pipelines import input_fn

In [2]:
X, y, set_range = add_ab()
X_extrapolated = x_generator(range_l=-20, range_u=20)

m, n, p, q = set_range
a = np.sum(X_extrapolated[:, m:n], axis=1)
b = np.sum(X_extrapolated[:, p:q], axis=1)

y_extrapolated = a + b

In [3]:
X.shape, y.shape, X_extrapolated.shape, y_extrapolated.shape

((10000, 100), (10000,), (10000, 100), (10000,))

In [4]:
y = np.expand_dims(y, 1)
y_extrapolated = np.expand_dims(y_extrapolated, 1)

In [5]:
def train_test_split(X, y, val_split=0.2):
    indices = np.random.permutation(X.shape[0])
    split = int(0.2 * X.shape[0])
    test_indices = indices[:split]
    train_indices = indices[split:]
    return X[train_indices], y[train_indices], X[test_indices], y[test_indices]

In [6]:
X_train, y_train, X_test, y_test = train_test_split(X, y)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 100), (2000, 100), (8000, 1), (2000, 1))

In [7]:
train_input_fn = lambda: input_fn(X_train, y_train, 2000, 32)
test_input_fn1 = lambda: input_fn(X_test, y_test, 2000, 32, is_train=False)
test_input_fn2 = lambda: input_fn(X_extrapolated, y_extrapolated, 2000, 32, is_train=False)

In [8]:
EPOCHS = 10
BATCH_SIZE = 32
TRAIN_STEPS = EPOCHS * X_train.shape[0] // BATCH_SIZE

TRAIN_STEPS

2500

In [9]:
from models.arithmetic_model import arithmetic_nac_fn
from models.arithmetic_model import arithmetic_nalu_fn

In [10]:
estimator = tf.estimator.Estimator(model_fn=arithmetic_nac_fn,
                                   params={}, model_dir='./ckpts/arithmetic_add/nac')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_task_id': 0, '_evaluation_master': '', '_is_chief': True, '_num_worker_replicas': 1, '_task_type': 'worker', '_save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000029149B1AF60>, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_train_distribute': None, '_model_dir': './ckpts/arithmetic_add/nac', '_master': '', '_session_config': None, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0}


In [11]:
estimator.train(input_fn=train_input_fn, steps=TRAIN_STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./ckpts/arithmetic_add/nac\model.ckpt.
INFO:tensorflow:step = 0, loss = 1494.7577
INFO:tensorflow:global_step/sec: 180.545
INFO:tensorflow:step = 100, loss = 1780.4465 (0.554 sec)
INFO:tensorflow:global_step/sec: 332.39
INFO:tensorflow:step = 200, loss = 1208.0111 (0.301 sec)
INFO:tensorflow:global_step/sec: 301.331
INFO:tensorflow:step = 300, loss = 1620.1523 (0.332 sec)
INFO:tensorflow:global_step/sec: 178.556
INFO:tensorflow:step = 400, loss = 667.18835 (0.560 sec)
INFO:tensorflow:global_step/sec: 315.925
INFO:tensorflow:step = 500, loss = 626.86365 (0.317 sec)
INFO:tensorflow:global_step/sec: 316.385
INFO:tensorflow:step = 600, loss = 371.2116 (0.332 sec)
INFO:tensorflow:global_step/sec: 333.238
INFO:ten

In [12]:
estimator.evaluate(input_fn=test_input_fn1)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-19-14:33:49
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ckpts/arithmetic_add/nac\model.ckpt-2500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-19-14:33:51
INFO:tensorflow:Saving dict for global step 2500: global_step = 2500, loss = 0.034050614


{'global_step': 2500, 'loss': 0.034050614}

In [13]:
estimator.evaluate(input_fn=test_input_fn2)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-19-14:33:54
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ckpts/arithmetic_add/nac\model.ckpt-2500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-19-14:33:56
INFO:tensorflow:Saving dict for global step 2500: global_step = 2500, loss = 0.13673519


{'global_step': 2500, 'loss': 0.13673519}

In [10]:
tf.reset_default_graph()
estimator = tf.estimator.Estimator(model_fn=arithmetic_nalu_fn,
                                   params={}, model_dir='./ckpts/arithmetic_add/nalu')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_tf_random_seed': None, '_is_chief': True, '_save_summary_steps': 100, '_task_type': 'worker', '_task_id': 0, '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001BCA11D9FD0>, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_session_config': None, '_keep_checkpoint_max': 5, '_model_dir': './ckpts/arithmetic_add/nalu', '_train_distribute': None, '_service': None, '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_master': '', '_evaluation_master': '', '_global_id_in_cluster': 0}


In [12]:
estimator.train(input_fn=train_input_fn, steps=TRAIN_STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ckpts/arithmetic_add/nalu\model.ckpt-2500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2501 into ./ckpts/arithmetic_add/nalu\model.ckpt.
INFO:tensorflow:loss = 2.8707652, step = 2500
INFO:tensorflow:global_step/sec: 106.828
INFO:tensorflow:loss = 1.3754387, step = 2600 (0.939 sec)
INFO:tensorflow:global_step/sec: 207.31
INFO:tensorflow:loss = 2.1910763, step = 2700 (0.498 sec)
INFO:tensorflow:global_step/sec: 152.666
INFO:tensorflow:loss = 5.6631956, step = 2800 (0.653 sec)
INFO:tensorflow:global_step/sec: 136.392
INFO:tensorflow:loss = 7.8654966, step = 2900 (0.718 sec)
INFO:tensorflow:global_step/sec: 145.739
INFO:tensorflow:loss = 0.5129818, step = 3000 (0.686 sec)
INFO:tensorflow:global_step/sec: 152.833
INFO:te

In [13]:
estimator.evaluate(input_fn=test_input_fn1)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-19-16:57:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ckpts/arithmetic_add/nalu\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-19-16:57:09
INFO:tensorflow:Saving dict for global step 5000: global_step = 5000, loss = 0.84773093


{'global_step': 5000, 'loss': 0.84773093}

In [14]:
estimator.evaluate(input_fn=test_input_fn2)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-19-16:57:14
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ckpts/arithmetic_add/nalu\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-19-16:57:17
INFO:tensorflow:Saving dict for global step 5000: global_step = 5000, loss = 3.2616096


{'global_step': 5000, 'loss': 3.2616096}

In [1]:
import numpy as np
import tensorflow as tf

from utils import x_generator
from utils.nonlins import *
from utils.pipelines import input_fn

In [2]:
X, y = relu()

X_extrapolated = x_generator(range_l=-20, range_u=20, dims=1)

y_extrapolated = np.maximum(X_extrapolated, 0)

In [3]:
X.shape, y.shape, X_extrapolated.shape, y_extrapolated.shape

((10000, 1), (10000, 1), (10000, 1), (10000, 1))

In [4]:
def train_test_split(X, y, val_split=0.2):
    indices = np.random.permutation(X.shape[0])
    split = int(0.2 * X.shape[0])
    test_indices = indices[:split]
    train_indices = indices[split:]
    return X[train_indices], y[train_indices], X[test_indices], y[test_indices]

In [5]:
X_train, y_train, X_test, y_test = train_test_split(X, y)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 1), (2000, 1), (8000, 1), (2000, 1))

In [6]:
train_input_fn = lambda: input_fn(X_train, y_train, 2000, 32)
test_input_fn1 = lambda: input_fn(X_test, y_test, 2000, 32, is_train=False)
test_input_fn2 = lambda: input_fn(X_extrapolated, y_extrapolated, 2000, 32, is_train=False)

In [7]:
EPOCHS = 10
BATCH_SIZE = 32
TRAIN_STEPS = EPOCHS * X_train.shape[0] // BATCH_SIZE

TRAIN_STEPS

2500

In [8]:
from models.nonlin_model import nonlin_nac_fn
from models.nonlin_model import nonlin_nalu_fn

In [9]:
estimator = tf.estimator.Estimator(model_fn=nonlin_nac_fn,
                                   params={}, model_dir='./ckpts/nonlin_relu/nac')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_train_distribute': None, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E9D7031B38>, '_num_ps_replicas': 0, '_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_session_config': None, '_service': None, '_is_chief': True, '_save_checkpoints_secs': 600, '_task_id': 0, '_model_dir': './ckpts/nonlin_relu/nac', '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_master': '', '_global_id_in_cluster': 0, '_evaluation_master': ''}


In [13]:
estimator.train(input_fn=train_input_fn, steps=TRAIN_STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ckpts/nonlin_relu/nac\model.ckpt-7500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 7501 into ./ckpts/nonlin_relu/nac\model.ckpt.
INFO:tensorflow:loss = 9.727577, step = 7500
INFO:tensorflow:global_step/sec: 262.816
INFO:tensorflow:loss = 9.245731, step = 7600 (0.379 sec)
INFO:tensorflow:global_step/sec: 282.608
INFO:tensorflow:loss = 8.997835, step = 7700 (0.354 sec)
INFO:tensorflow:global_step/sec: 249.38
INFO:tensorflow:loss = 8.293657, step = 7800 (0.401 sec)
INFO:tensorflow:global_step/sec: 301.23
INFO:tensorflow:loss = 8.180317, step = 7900 (0.348 sec)
INFO:tensorflow:global_step/sec: 231.28
INFO:tensorflow:loss = 10.652992, step = 8000 (0.433 sec)
INFO:tensorflow:global_step/sec: 202.116
INFO:tensorflow:loss =

In [14]:
estimator.evaluate(input_fn=test_input_fn1)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-19-17:03:58
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ckpts/nonlin_relu/nac\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-19-17:03:59
INFO:tensorflow:Saving dict for global step 10000: global_step = 10000, loss = 8.477541


{'global_step': 10000, 'loss': 8.477541}

In [15]:
estimator.evaluate(input_fn=test_input_fn2)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-19-17:04:04
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ckpts/nonlin_relu/nac\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-19-17:04:05
INFO:tensorflow:Saving dict for global step 10000: global_step = 10000, loss = 33.531742


{'global_step': 10000, 'loss': 33.531742}

In [16]:
tf.reset_default_graph()
estimator = tf.estimator.Estimator(model_fn=nonlin_nalu_fn,
                                   params={}, model_dir='./ckpts/nonlin_relu/nalu')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_train_distribute': None, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E9D701F828>, '_num_ps_replicas': 0, '_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_session_config': None, '_service': None, '_save_checkpoints_secs': 600, '_evaluation_master': '', '_task_id': 0, '_model_dir': './ckpts/nonlin_relu/nalu', '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_master': '', '_global_id_in_cluster': 0, '_is_chief': True}


In [17]:
estimator.train(input_fn=train_input_fn, steps=TRAIN_STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./ckpts/nonlin_relu/nalu\model.ckpt.
INFO:tensorflow:loss = 24.72435, step = 0
INFO:tensorflow:global_step/sec: 136.526
INFO:tensorflow:loss = 17.695835, step = 100 (0.748 sec)
INFO:tensorflow:global_step/sec: 182.394
INFO:tensorflow:loss = 8.527319, step = 200 (0.552 sec)
INFO:tensorflow:global_step/sec: 166.623
INFO:tensorflow:loss = 2.3056095, step = 300 (0.581 sec)
INFO:tensorflow:global_step/sec: 145.883
INFO:tensorflow:loss = 1.8077557, step = 400 (0.685 sec)
INFO:tensorflow:global_step/sec: 152.672
INFO:tensorflow:loss = 1.8797323, step = 500 (0.655 sec)
INFO:tensorflow:global_step/sec: 125.32
INFO:tensorflow:loss = 0.67154175, step = 600 (0.798 sec)
INFO:tensorflow:global_step/sec: 142.432
INFO:tenso

In [18]:
estimator.evaluate(input_fn=test_input_fn1)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-19-17:05:29
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ckpts/nonlin_relu/nalu\model.ckpt-2500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-19-17:05:30
INFO:tensorflow:Saving dict for global step 2500: global_step = 2500, loss = 0.028774979


{'global_step': 2500, 'loss': 0.028774979}

In [19]:
estimator.evaluate(input_fn=test_input_fn2)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-19-17:05:36
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ckpts/nonlin_relu/nalu\model.ckpt-2500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-19-17:05:37
INFO:tensorflow:Saving dict for global step 2500: global_step = 2500, loss = 22.099894


{'global_step': 2500, 'loss': 22.099894}